In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


'''
The titanic data set contains three tables 
(1) train - the training data containing informations for all the passengers: Name, PassengerClass (Pclass), Sex, Age, SibSp, Parch, Ticket Number (Ticket), Fare, Cabin, Port of Embarkment (Embarked)
(2) gender - example prediction output where all the instances are female passengers
(3) test - same format as the training data, but used for 
'''

test = pd.read_csv('data/test.csv')
gender = pd.read_csv('data/gender_submission.csv')
train = pd.read_csv('data/train.csv')


In [43]:
'''
EDA + cleaning on the train data 
''' 

# train dataset contains the Name, Sex, Age, etc, basic information 
# print(train.info())

'''
check for NaN (null) data by columns
'''
# Cabin and Age have a lot of Null values 

'''
because Cabin has poor data quality and we are not interested in Embarked, we can just drop them
'''
# table without Cabin and Embarked 
dropped = train.drop(['Cabin', 'Embarked'], axis=1)

# another version of the table without invalid Age value 
filtered_age = dropped[dropped['Age'].notnull()]
print(filtered_age.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  714 non-null    int64  
 1   Survived     714 non-null    int64  
 2   Pclass       714 non-null    int64  
 3   Name         714 non-null    object 
 4   Sex          714 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        714 non-null    int64  
 7   Parch        714 non-null    int64  
 8   Ticket       714 non-null    object 
 9   Fare         714 non-null    float64
dtypes: float64(2), int64(5), object(3)
memory usage: 53.0+ KB
None


In [48]:
'''
build classifier
'''
# first define the features 
# we are interested in the following features: Sex, Age, passenger class, fare
# EX: [[(Sex), (Age), (Pclass), (Fare)], ...]
features = filtered_age[['Sex', 'Age', 'Pclass', 'Fare']].to_numpy()


array([['male', 22.0, 3, 7.25],
       ['female', 38.0, 1, 71.2833],
       ['female', 26.0, 3, 7.925],
       ...,
       ['female', 19.0, 1, 30.0],
       ['male', 26.0, 1, 30.0],
       ['male', 32.0, 3, 7.75]], dtype=object)